In [14]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
%matplotlib inline

In [22]:
def get_norm_ll_vec(x, mu, sigma):
    mu = tf.reshape(mu, (1, ))
    sigma = tf.reshape(sigma, (1, ))
    return \
        -0.5 * (x - mu) ** 2 / (sigma ** 2) \
        - tf.math.log(sigma)


def get_norm_ll(x, mu, sigma):
    return tf.reduce_sum(get_norm_ll_vec(x, mu, sigma))


def get_norm_ll_infl(x, mu, sigma, w):
    return tf.reduce_sum(w * get_norm_ll_vec(x, mu, sigma))


In [27]:
num_obs = 10
mu = 2.0
sigma = 1.5
x = np.random.normal(loc=mu, scale=sigma, size=num_obs)
w = np.ones(num_obs)

In [28]:
get_norm_ll_infl(x, mu, sigma, w)

<tf.Tensor: shape=(), dtype=float32, numpy=-12.113289>

In [50]:
num_clusters = 3
mu_vec = np.linspace(1, num_clusters, num=num_clusters)
sigma_vec = np.linspace(0.1, 0.5, num=num_clusters)
pi_vec = np.linspace(1, num_clusters, num=num_clusters)
pi_vec = pi_vec / np.sum(pi_vec)

def get_norm_clustering_mstep_mat(x, mu_vec, sigma_vec, pi_vec, e_z):
    ll_mat = tf.stack(
        [ get_norm_ll_vec(x, mu_vec[k], sigma_vec[k]) \
          for k in range(num_clusters) ], axis=1)

    prior_mat = tf.math.log(tf.expand_dims(pi_vec, 0) * e_z)
    
    return ll_mat + prior_mat

def get_norm_clustering_mstep(x, mu_vec, sigma_vec, pi_vec, e_z):
    return tf.reduce_sum(get_norm_clustering_mstep_mat(x, mu_vec, sigma_vec, pi_vec, e_z))

e_z = np.random.random((num_obs, num_clusters))
get_norm_clustering_mstep(x, mu_vec, sigma_vec, pi_vec, e_z)

<tf.Tensor: shape=(), dtype=float64, numpy=-2991.819729986609>

In [67]:
def get_logistic_ll_vec(y, x_mat, theta):
    z = x_mat @ theta
    return y * z + tf.math.log1p(z)

def get_logistic_ll(y, x_mat, theta):
    return tf.reduce_sum(get_logistic_ll_vec(y, x_mat, theta))


dim = 3
x_mat = np.hstack([ np.ones((num_obs, 1)), np.random.random((num_obs, dim - 1)) ])
theta = np.random.random(dim) / 20
y = (np.random.random(num_obs) < x_mat @ theta).astype(np.float64)

get_logistic_ll(y, x_mat, theta)

<tf.Tensor: shape=(), dtype=float64, numpy=0.6724617357284832>